Using a list of EANs to search in Continente Online and bring product's information

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import json

In [20]:
# loading the barcode dataset
codigos = pd.read_csv("codigos_scrape_continente.csv")

In [21]:
len(codigos)

14422

## Defining scraping function

In [3]:
def scrape_continente(ean):
    url = f"https://www.continente.pt/pesquisa/?q={ean}&start=0&srule=Continente&pmin=0.01"

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    result = soup.select(".search-keyword-title")
    
    if response.status_code == 200:
        if result:
            keyword_value = int(result[0].find(class_="keyword").get_text())
            if keyword_value > 0:
                data = {'nome': [], 'link_produto': [], 'embalagem': [], 'ean': [], 'desconto': [], 'pvp_recomendado': [], 'preco_por_volume': [], 'promocao': [], 'link_imagem': [], 'produtos': []}

                nome_produto_tag = soup.find("a", {"class": "pwc-tile--description"})
                link_produto_tag = soup.find("a", {"class": "pwc-tile--description"})
                embalagem_tag = soup.find("p", {"class": "pwc-tile--quantity"})
                desconto_tag = soup.find("span", {"class": "ct-product-tile-badge-value--pvpr"})
                pvp_recomendado_tag = soup.find("p", {"class": "pwc-discount-amount"})
                preco_por_volume_tag = soup.find("span", {"class": "ct-price-value"})
                promocao_tag = soup.find("div", {"class": "ct-product-tile-badge-value-wrapper"})
                link_imagem_tag = soup.find("img", {"class": "ct-tile-image"})
                produtos_tag = soup.find("div", {"class": "product-tile"})

                nome_produto = nome_produto_tag.text.strip() if nome_produto_tag else None
                link_produto = link_produto_tag['href'] if link_produto_tag else None
                embalagem = embalagem_tag.text.strip() if embalagem_tag else None
                desconto_pct = desconto_tag.text.strip() if desconto_tag else "0"
                pvp_recomendado = pvp_recomendado_tag.get_text().replace("PVP Recomendado: ","") if pvp_recomendado_tag else None
                preco_por_volume = preco_por_volume_tag.text.strip() if preco_por_volume_tag else None
                promocao = promocao_tag.text.strip() if promocao_tag else None
                link_imagem = link_imagem_tag['data-src'] if link_imagem_tag else None
                produtos = produtos_tag['data-product-tile-impression'] if produtos_tag else None

                data['nome'].append(nome_produto)
                data['link_produto'].append(link_produto)
                data['embalagem'].append(embalagem)
                data['ean'].append(ean)
                data['desconto'].append(desconto_pct)
                data['pvp_recomendado'].append(pvp_recomendado)
                data['preco_por_volume'].append(preco_por_volume)
                data['promocao'].append(promocao)
                data['link_imagem'].append(link_imagem)
                data['produtos'].append(produtos)

                df_produto = pd.DataFrame(data)
                return df_produto
            else:
                print("Produto não encontrado")
        else:
            print("Produto não encontrado")
    else:
        print('Erro ao acessar o site:', response.status_code)

## Scraping products

In [ ]:
df_produtos = pd.DataFrame()

for ean in codigos['BARCODE']:
    print(ean)
    aux = scrape_continente(str(ean))
    df_produtos = pd.concat([df_produtos, aux], ignore_index=True)

# Saving results in csv and pickle

df_produtos.to_csv(f"{pd.Timestamp.now().strftime('%Y%m%d')}_scrape_continente.csv", index=False)
df_produtos.to_pickle(f"{pd.Timestamp.now().strftime('%Y%m%d')}_produtos.pkl")

In [5]:
# loading pickle's file

df_produtos_pickle = "20240215_produtos.pkl"
df_produtos = pd.read_pickle(df_produtos_pickle)

df_produtos

,nome,link_produto,embalagem,ean,desconto,pvp_recomendado,preco_por_volume,promocao,link_imagem,produtos
0,Máscara Cabelo Elvive Reconstrutora Dream Long,https://www.continente.pt/produto/mascara-cabe...,emb. 300 ml,3600523587315,30,"€6,99/un","€16,30",30\n\n\n%,https://www.continente.pt/dw/image/v2/BDVS_PRD...,"{""name"":""Máscara Cabelo Elvive Reconstrutora D..."
1,Condicionador Ultra Suave Óleos de Argão e Cam...,https://www.continente.pt/produto/condicionado...,emb. 400 ml,3600541575790,35,"€5,59/un","€9,07",35\n\n\n%,https://www.continente.pt/dw/image/v2/BDVS_PRD...,"{""name"":""Condicionador Ultra Suave Óleos de Ar..."
2,Condicionador Profissional Hidratação Intensa,https://www.continente.pt/produto/condicionado...,emb. 900 ml,5601312602217,0,None,"€2,21",None,https://www.continente.pt/dw/image/v2/BDVS_PRD...,"{""name"":""Condicionador Profissional Hidratação..."
3,Gel Higiene Íntima Suave,https://www.continente.pt/produto/gel-higiene-...,emb. 400 ml,3468080408494,0,None,"€18,73",None,https://www.continente.pt/dw/image/v2/BDVS_PRD...,"{""name"":""Gel Higiene Íntima Suave"",""id"":""67379..."
4,Gel Higiene Íntima Suave,https://www.continente.pt/produto/gel-higiene-...,emb. 250 ml,3468080408258,0,None,"€21,16",None,https://www.continente.pt/dw/image/v2/BDVS_PRD...,"{""name"":""Gel Higiene Íntima Suave"",""id"":""66406..."
...,...,...,...,...,...,...,...,...,...,...
10278,Condicionador Fructis Hair Food Ananás,https://www.continente.pt/produto/condicionado...,emb. 350 ml,3600542486668,40,"€6,59/un","€11,29",40\n\n\n%,https://www.continente.pt/dw/image/v2/BDVS_PRD...,"{""name"":""Condicionador Fructis Hair Food Ananá..."
10279,Champô Elvive Premium Bond Repair,https://www.continente.pt/produto/champo-elviv...,emb. 200 ml,3600524074685,25,"€6,65/un","€24,90",25\n\n\n%,https://www.continente.pt/dw/image/v2/BDVS_PRD...,"{""name"":""Champô Elvive Premium Bond Repair"",""i..."
10280,Cápsulas de Café Au Lait Int 7,https://www.continente.pt/produto/capsulas-de-...,emb. 30 un,7613034365774,0,None,"€0,33",None,https://www.continente.pt/dw/image/v2/BDVS_PRD...,"{""name"":""Cápsulas de Café Au Lait Int 7"",""id"":..."
10281,Vegegurte Crema Catalana,https://www.continente.pt/produto/vegegurte-cr...,emb. 135 gr,8424790274014,0,None,"€12,81",None,https://www.continente.pt/dw/image/v2/BDVS_PRD...,"{""name"":""Vegegurte Crema Catalana"",""id"":""76313..."


## Cleaning data

In [6]:
df_produtos['promocao'] = df_produtos['promocao'].str.replace('\n%', '').str.strip()

# sales flag
df_produtos['promocao_flag'] = df_produtos['desconto'].apply(lambda x: 1 if x > 0 else 0)

# transforming in numerical pvp_recomendado
df_produtos['pvp_recomendado'] = df_produtos['pvp_recomendado'].str.replace('€', '').str.replace('/un', '').str.replace(',', '.')
df_produtos['pvp_recomendado'] = pd.to_numeric(df_produtos['pvp_recomendado'], errors='coerce')

# transforming in numerical preco_por_volume
df_produtos['preco_por_volume'] = df_produtos['preco_por_volume'].str.replace('€', '').str.replace(',', '.')
df_produtos['preco_por_volume'] = pd.to_numeric(df_produtos['preco_por_volume'], errors='coerce')

In [7]:
# transforming into columns the 'product'
def parse_json(json_str):
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        return {}  

# converting the column 'produtos' to dictionaries
df_produtos['produtos'] = df_produtos['produtos'].apply(parse_json)

# creating new columns based on the dictionary
df_produtos['product_name'] = df_produtos['produtos'].apply(lambda x: x.get('name', None))
df_produtos['product_id'] = df_produtos['produtos'].apply(lambda x: x.get('id', None))
df_produtos['product_price'] = df_produtos['produtos'].apply(lambda x: x.get('price', None))
df_produtos['product_brand'] = df_produtos['produtos'].apply(lambda x: x.get('brand', None))
df_produtos['product_category'] = df_produtos['produtos'].apply(lambda x: x.get('category', None))
df_produtos['product_variant'] = df_produtos['produtos'].apply(lambda x: x.get('variant', None))
df_produtos['product_channel'] = df_produtos['produtos'].apply(lambda x: x.get('channel', None))

# transforming discount into percentage
df_produtos["desconto"] = pd.to_numeric(df_produtos["desconto"], errors='coerce')/100

In [16]:
# Dataset final
df_produtos

,nome,link_produto,embalagem,ean,desconto,pvp_recomendado,preco_por_volume,promocao,link_imagem,produtos,promocao_flag,product_name,product_id,product_price,product_brand,product_category,product_variant,product_channel
0,Máscara Cabelo Elvive Reconstrutora Dream Long,https://www.continente.pt/produto/mascara-cabe...,emb. 300 ml,3600523587315,0.30,6.99,16.30,30,https://www.continente.pt/dw/image/v2/BDVS_PRD...,{'name': 'Máscara Cabelo Elvive Reconstrutora ...,1,Máscara Cabelo Elvive Reconstrutora Dream Long,6421142,4.89,Elvive LOréal Paris,Beleza e Higiene/Cabelo/Máscaras e Tratamentos,,col
1,Condicionador Ultra Suave Óleos de Argão e Cam...,https://www.continente.pt/produto/condicionado...,emb. 400 ml,3600541575790,0.35,5.59,9.07,35,https://www.continente.pt/dw/image/v2/BDVS_PRD...,{'name': 'Condicionador Ultra Suave Óleos de A...,1,Condicionador Ultra Suave Óleos de Argão e Cam...,5365191,3.63,Ultra Suave Garnier,Beleza e Higiene/Cabelo/Condicionadores,,col
2,Condicionador Profissional Hidratação Intensa,https://www.continente.pt/produto/condicionado...,emb. 900 ml,5601312602217,0.00,NaN,2.21,None,https://www.continente.pt/dw/image/v2/BDVS_PRD...,{'name': 'Condicionador Profissional Hidrataçã...,0,Condicionador Profissional Hidratação Intensa,6855958,1.99,MyLabel,Beleza e Higiene/Cabelo/Condicionadores,,col
3,Gel Higiene Íntima Suave,https://www.continente.pt/produto/gel-higiene-...,emb. 400 ml,3468080408494,0.00,NaN,18.73,None,https://www.continente.pt/dw/image/v2/BDVS_PRD...,"{'name': 'Gel Higiene Íntima Suave', 'id': '67...",0,Gel Higiene Íntima Suave,6737977,7.49,Corine de Farme,Beleza e Higiene/Higiene Íntima/Gel e Sabonete...,,col
4,Gel Higiene Íntima Suave,https://www.continente.pt/produto/gel-higiene-...,emb. 250 ml,3468080408258,0.00,NaN,21.16,None,https://www.continente.pt/dw/image/v2/BDVS_PRD...,"{'name': 'Gel Higiene Íntima Suave', 'id': '66...",0,Gel Higiene Íntima Suave,6640649,5.29,Corine de Farme,Beleza e Higiene/Higiene Íntima/Gel e Sabonete...,,col
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10278,Condicionador Fructis Hair Food Ananás,https://www.continente.pt/produto/condicionado...,emb. 350 ml,3600542486668,0.40,6.59,11.29,40,https://www.continente.pt/dw/image/v2/BDVS_PRD...,{'name': 'Condicionador Fructis Hair Food Anan...,1,Condicionador Fructis Hair Food Ananás,7654893,3.95,Fructis Garnier,Beleza e Higiene/Cabelo/Condicionadores,,col
10279,Champô Elvive Premium Bond Repair,https://www.continente.pt/produto/champo-elviv...,emb. 200 ml,3600524074685,0.25,6.65,24.90,25,https://www.continente.pt/dw/image/v2/BDVS_PRD...,"{'name': 'Champô Elvive Premium Bond Repair', ...",1,Champô Elvive Premium Bond Repair,7698093,4.98,Elvive LOréal Paris,Beleza e Higiene/Cabelo/Champôs,,col
10280,Cápsulas de Café Au Lait Int 7,https://www.continente.pt/produto/capsulas-de-...,emb. 30 un,7613034365774,0.00,NaN,0.33,None,https://www.continente.pt/dw/image/v2/BDVS_PRD...,"{'name': 'Cápsulas de Café Au Lait Int 7', 'id...",0,Cápsulas de Café Au Lait Int 7,5460510,9.99,Dolce Gusto,"Mercearia/Café, Chá e Chocolate Solúvel/Café e...",,col
10281,Vegegurte Crema Catalana,https://www.continente.pt/produto/vegegurte-cr...,emb. 135 gr,8424790274014,0.00,NaN,12.81,None,https://www.continente.pt/dw/image/v2/BDVS_PRD...,"{'name': 'Vegegurte Crema Catalana', 'id': '76...",0,Vegegurte Crema Catalana,7631306,1.73,Pastoret,Laticínios e Ovos/Iogurtes/Vegegurtes e Yofu,,col


In [10]:
df_produtos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10283 entries, 0 to 10282
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   nome              10283 non-null  object 
 1   link_produto      10283 non-null  object 
 2   embalagem         10283 non-null  object 
 3   ean               10283 non-null  object 
 4   desconto          10283 non-null  float64
 5   pvp_recomendado   3756 non-null   float64
 6   preco_por_volume  10272 non-null  float64
 7   promocao          3918 non-null   object 
 8   link_imagem       10283 non-null  object 
 9   produtos          10283 non-null  object 
 10  promocao_flag     10283 non-null  int64  
 11  product_name      10247 non-null  object 
 12  product_id        10247 non-null  object 
 13  product_price     10247 non-null  float64
 14  product_brand     10247 non-null  object 
 15  product_category  10247 non-null  object 
 16  product_variant   10247 non-null  object

In [17]:
# saving the final results
df_produtos.to_csv(f"{pd.Timestamp.now().strftime('%Y%m%d')}_scrape_continente_clean.csv", index=False)
df_produtos.to_pickle(f"{pd.Timestamp.now().strftime('%Y%m%d')}_produtos_clean.pkl")